In [1]:
from sklearn.preprocessing import Normalizer
import pandas as pd
import numpy as np

df = pd.read_csv("star_classification.csv")
df = df.sample(400)


df.loc[df['class'] == "GALAXY", 'class'] = 1
df.loc[df['class'] == "QSO", 'class'] = 2
df.loc[df['class'] == "STAR", 'class'] = 3
df['class'] = df['class'].astype(int)
y = df['class']
print(df['class'].value_counts())

df = df.drop(columns=['obj_ID', 'run_ID', 'field_ID', 'spec_obj_ID', 'fiber_ID'])
df = df.astype('float') 
# нормализация
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df[df.columns] = scaler.fit_transform(df[df.columns])
#df = (df-df.min())/(df.max()-df.min())
df.info()
classes = set(y)
print(df.head(5))
# 3 класса по дороговизне
classes

1    221
3     94
2     85
Name: class, dtype: int64
<class 'pandas.core.frame.DataFrame'>
Int64Index: 400 entries, 88073 to 63199
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   alpha     400 non-null    float64
 1   delta     400 non-null    float64
 2   u         400 non-null    float64
 3   g         400 non-null    float64
 4   r         400 non-null    float64
 5   i         400 non-null    float64
 6   z         400 non-null    float64
 7   rerun_ID  400 non-null    float64
 8   cam_col   400 non-null    float64
 9   class     400 non-null    float64
 10  redshift  400 non-null    float64
 11  plate     400 non-null    float64
 12  MJD       400 non-null    float64
dtypes: float64(13)
memory usage: 43.8 KB
          alpha     delta         u         g         r         i         z  \
88073  0.041707  0.226142  0.704831  0.819563  0.775600  0.729915  0.656043   
90979  0.977813  0.548464  0.807882  0.625250  0.71

{1, 2, 3}

In [2]:
df['y_true'] = y
bts = []
for i in range(3):
  df_bt = df.sample(frac=0.5)
  y_bt = df_bt['y_true']
  X_bt = df_bt.drop(columns=['y_true'])
  bts.append([X_bt, y_bt])

In [3]:
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

models = [GaussianNB(), DecisionTreeClassifier(), LinearSVC()]
preds= []
for i, model in enumerate(models):
  X_bt = bts[i][0]
  y_bt = bts[i][1]
  model.fit(X_bt, y_bt)
  y_pred = model.predict(X_bt)
  pred_df = pd.DataFrame(y_pred, index = X_bt.index)
  preds.append(pred_df)
  print(accuracy_score(y_bt, y_pred))

1.0
1.0
0.995


In [4]:
preds_df = pd.concat(preds, axis=1)
preds_df.head()

,0,0,0
149,3.0,NaN,NaN
613,NaN,NaN,1.0
1901,2.0,NaN,NaN
2046,NaN,1.0,1.0
2822,3.0,3.0,3.0


In [5]:
preds_df.to_csv('preds.csv')

In [6]:
!pip install owlready2
from owlready2 import *

onto = get_ontology('onto.owl')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
import types

with onto:
  tsr = types.new_class('tsr', (ObjectProperty, TransitiveProperty, SymmetricProperty, ReflexiveProperty))
  ts = types.new_class('ts', (ObjectProperty, TransitiveProperty, SymmetricProperty))

  for cl in classes:
    onto_cl = types.new_class(f'Class{cl}', (Thing,))
    onto_cl_know = types.new_class(f'Know_Class{cl}', (Thing,))
    onto_cl_know.equivalent_to.append(tsr.only(onto_cl))
    onto_cl_believe = types.new_class(f'Believe_Class{cl}', (Thing,))
    onto_cl_believe.equivalent_to.append(ts.only(onto_cl))

onto.save('onto.owl', format='ntriples')

In [8]:
with onto:
  for index, row in preds_df.iterrows():
    inst_old = None
    for j, pred in enumerate(preds_df.columns):
      cl = row.iloc[j]
      if not pd.isna(cl):
        cl = int(cl)
        onto_cl = onto[f'Class{cl}']
        inst_new = onto_cl(f'inst{index}_{j}')
        if inst_old != None:
          inst_old.tsr.append(inst_new)
          inst_old.ts.append(inst_new)
        inst_old = inst_new

onto.save('onto_with_instances.owl', format='ntriples')

In [9]:
with onto:
  sync_reasoner_pellet(infer_property_values=True, debug=2)

onto.save('onto_with_relations.owl', format='ntriples')

* Owlready2 * Running Pellet...
    java -Xmx2000M -cp /usr/local/lib/python3.8/dist-packages/owlready2/pellet/owlapi-distribution-3.4.3-bin.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/jena-iri-0.9.5.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/jena-tdb-0.10.0.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/jcl-over-slf4j-1.6.4.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/jena-arq-2.10.0.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/aterm-java-1.6.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/xml-apis-1.4.01.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/slf4j-api-1.6.4.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/commons-codec-1.6.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/pellet-2.3.1.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/httpcore-4.2.2.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/xercesImpl-2.10.0.jar:/usr/local/lib/p

* Owlready * Adding relation onto.inst51086_1 tsr onto.inst51086_1
* Owlready * Adding relation onto.inst98573_0 tsr onto.inst98573_0
* Owlready * Adding relation onto.inst98573_0 ts onto.inst98573_0
* Owlready * Adding relation onto.inst22513_2 tsr onto.inst22513_2
* Owlready * Adding relation onto.inst22513_2 ts onto.inst22513_2
* Owlready * Adding relation onto.inst57245_0 tsr onto.inst57245_0
* Owlready * Adding relation onto.inst57245_0 ts onto.inst57245_0
* Owlready * Adding relation onto.inst5510_0 tsr onto.inst5510_0
* Owlready * Adding relation onto.inst5510_0 tsr onto.inst5510_2
* Owlready * Adding relation onto.inst5510_0 ts onto.inst5510_0
* Owlready * Adding relation onto.inst5510_0 ts onto.inst5510_2
* Owlready * Adding relation onto.inst37103_2 tsr onto.inst37103_2
* Owlready * Adding relation onto.inst37103_2 ts onto.inst37103_2
* Owlready * Adding relation onto.inst44430_1 tsr onto.inst44430_1
* Owlready * Adding relation onto.inst44430_1 ts onto.inst44430_1
* Owlready

* Owlready * (NB: only changes on entities loaded in Python are shown, other changes are done but not listed)


In [10]:
with onto:
  for inst in onto.individuals():
    close_world(inst)

# onto.save('onto_cwa.owl', format='ntriples')

In [11]:
with onto:
  sync_reasoner_pellet(debug=2)

onto.save('onto_reasoned.owl', format='ntriples')

* Owlready2 * Running Pellet...
    java -Xmx2000M -cp /usr/local/lib/python3.8/dist-packages/owlready2/pellet/owlapi-distribution-3.4.3-bin.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/jena-iri-0.9.5.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/jena-tdb-0.10.0.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/jcl-over-slf4j-1.6.4.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/jena-arq-2.10.0.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/aterm-java-1.6.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/xml-apis-1.4.01.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/slf4j-api-1.6.4.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/commons-codec-1.6.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/pellet-2.3.1.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/httpcore-4.2.2.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/xercesImpl-2.10.0.jar:/usr/local/lib/p